# This notebook analyzes where oldtimers on TRP go after the quarantine? 

In [1]:
# -*- coding: utf-8 -*-

In [2]:
import re
import os
import sys
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
from scipy import stats
import csv
import pprint
from collections import defaultdict
import time
import datetime
import math
import glob
import praw
# import matplotlib.pyplot as plt; plt.rcdefaults()
# from IPython.display import Markdown, display

In [3]:
pd.options.display.max_rows = 1999

In [4]:
QUARANTINE_DATE = "28/09/2018"
quarDataUnix = time.mktime(datetime.datetime.strptime(QUARANTINE_DATE, "%d/%m/%Y").timetuple())

In [5]:
USER_HISTORY_COMMENTS = "/mnt/storage/quarantine/data/TRP_users_comments"
USER_HISTORY_SUBMISSIONS = "/mnt/storage/quarantine/data/TRP_users_submissions.csv"
TOTAL_USERS = "/mnt/storage/quarantine/data/candidate_control_subs_unique_users.csv"

In [6]:
all_files = glob.glob(USER_HISTORY_COMMENTS + "/*.csv")

li = []

# Reading in comments
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, usecols = ["created_utc", "subreddit", "author"])
    li.append(df)
    
# Reading in submissions
df = pd.read_csv(USER_HISTORY_SUBMISSIONS, index_col=None, header=0, usecols = ["created_utc", "subreddit", "author"])
li.append(df)    

frame = pd.concat(li, axis=0, ignore_index=True)

/nethome/sjhaver3/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


## Tracking at the Post-level

In [7]:
pastPosts = frame[frame['created_utc'] <= quarDataUnix]
futurePosts = frame[frame['created_utc'] > quarDataUnix]

In [8]:
pastSubreddits = pastPosts.groupby('subreddit').size().reset_index(name='past_counts')

In [9]:
futureSubreddits = futurePosts.groupby('subreddit').size().reset_index(name='future_counts')

In [10]:
mergedDf = pd.merge(pastSubreddits, futureSubreddits, on='subreddit', how='inner')

In [11]:
mergedDf['Increase'] = mergedDf['future_counts']/mergedDf['past_counts']

In [12]:
g = mergedDf[mergedDf.past_counts >= 10].sort_values(by=['Increase'], ascending=False)[:100]
g

,subreddit,past_counts,future_counts,Increase
15323,cta,14,3670,262.142857
16751,funkoswap,34,6626,194.882353
17061,gofundme,43,6581,153.046512
10679,SmallYTChannel,67,7892,117.791045
840,AquaSwap,11,1249,113.545455
3625,DrawMeNSFW,15,1584,105.600000
15045,comedyheaven,140,13576,96.971429
22134,swingersr4r,22,1843,83.772727
13577,appliancerepair,12,990,82.500000
772,AnthemTheGame,173,12303,71.115607


## Tracking at the User-level

In [13]:
kPast = pastPosts.groupby(['subreddit', 'author']).size().to_frame(name = 'size').reset_index()
kFuture = futurePosts.groupby(['subreddit', 'author']).size().to_frame(name = 'size').reset_index()

In [14]:
kPastSubs = kPast.groupby('subreddit').size().reset_index(name='past_TRP_author_counts')
kFutureSubs = kFuture.groupby('subreddit').size().reset_index(name='future_TRP_author_counts')

In [15]:
mergedDf = pd.merge(kPastSubs, kFutureSubs, on='subreddit', how='inner')
mergedDf['Increase'] = mergedDf['future_TRP_author_counts']/mergedDf['past_TRP_author_counts']

In [16]:
g = mergedDf[mergedDf.past_TRP_author_counts >= 10].sort_values(by=['Increase'], ascending=False)[:100]
g

,subreddit,past_TRP_author_counts,future_TRP_author_counts,Increase
6077,Instagramreality,10,140,14.000000
772,AnthemTheGame,11,85,7.727273
20824,reddeadredemption,42,260,6.190476
598,AmItheAsshole,103,633,6.145631
16437,fightporn,31,184,5.935484
12071,TrueDetective,10,51,5.100000
17719,ihadastroke,10,47,4.700000
21664,smuggies,14,60,4.285714
19917,orthotropics,12,49,4.083333
3766,ENLIGHTENEDCENTRISM,29,110,3.793103
